# CSP Features 

### Steps of FB-CSP Classification 
(Fast EEG-Based Decoding Of The Directional Focus Of Auditory Attention Using Common Spatial Patterns)
- Step 1: Filter Bank
    - 14(B is number of filter bank) 8th-order butterworth filters. The first filter corresponds to frequency band 1-4 hz. The second with 2-6. And continuous with overlap 2 hz until 26(2+12*2)-30 hz. 
    - Downsample to 64 hz. (先不做)
- Step 2: CSP
    - How to choice the filter? RMOE (in this paper) vs. Covariance (mne, or common methods)
    - Choice k components from CSP filter
- Step 3: Log Energy
    - Summation the CSP pattern and calculate the log energy of every band and k filters
    - Output feature: Filter Bank B x k components
    - This step will be done by mne.decoding.csp.
- Step 4: Classification  
    - lda, svm

### Data pipline or workflow
Theres two task of the works. Binary classification vs. Multi-class
First in single classes
- Data Prepration: 
    - Input Data: 2(class) x 25(subjects) x 32/2(trials, unknown?) x channel num x trials timepoint
    - Output Data: channel num x 2(class) x segments numbers[subjects x trials x decision windows numbers] x decision windows size (4sec x 128hz)
- Step 1: Filter Bank
    - Input Data: (channel num, 2 x segments numbers, 512)
    - Ouput Data: 2, segments numbers, 14 bank num, 512
- Step 2: CSP
    - Input Data: (s)

### Step 1 - Data Prepration

In [1]:
import mne
import h5py
import numpy as np
import os
import scipy.io
import pandas as pd

# Variables
window_sec = 1
choice_direction = 90 # 0: 全選
bands = {
    '1': (12, 16),
    '2': (14, 18),
    '3': (16, 20),
    '4': (18, 22),
}
# for iter_ in range(4, 15):
#     pre_low, pre_high = bands[str(iter_-1)]
#     bands[str(iter_)] = (pre_low+2, pre_high+2)


# 使用範例
path = "./NJUNCA_preprocessed_arte_removed/"
expinfo_path = path + "expinfomat_csvs/"
mat_files = [f for f in os.listdir(path) if f.endswith('.mat')]
print(mat_files)    # 前兩個是info資料從第三個開始讀

subjects = 21
fs = 128  # sampling frequency
window_size = fs * window_sec  # 512
channels = 32  # assume full 32 channels
count = 0
side_dict = {
    "right": 0,
    "left": 1,
}


# Load data
all_segments = []
all_labels = []
all_trials = []
pre_trial_idx = 0

for subj in range(2, subjects):
    data_path = path + mat_files[subj]
    file = h5py.File(data_path, 'r')
    csv_path = expinfo_path + mat_files[subj].replace('.mat', '.csv')
    df = pd.read_csv(csv_path)['attended_lr']

    ref_data = file['data']
    ref_eeg = ref_data['eeg']
    ref_leftangle = ref_data['event']['leftWav']
    ref_rightangle = ref_data['event']['rightWav']
    ref_attenside = ref_data['event']['eeg']
    trials = len(ref_eeg[:])

    for trial in range(trials):
        try:
            left_angel_reg = file[ref_leftangle['value'][trial][0]]
            left_angel = file[left_angel_reg[0][0]][0][0]
            right_angel_reg = file[ref_rightangle['value'][trial][0]]
            right_angel = file[right_angel_reg[0][0]][0][0]

            if (left_angel*-1) != right_angel:
                print(f"{mat_files[subj]}-{trial}: {left_angel}/{right_angel} not the same angle ")
                continue
            # if abs(right_angel) != 90:
            #     print(f"{mat_files[subj]}-{trial}: {left_angel}/{right_angel} not 90 degree")
            #     continue
            print(f"{mat_files[subj]}-{trial}: {left_angel}/{right_angel}")
            ref = ref_eeg[trial][0]
            eeg_data = np.array(file[ref][:]) # shape: 32, time
            trial_len = eeg_data.shape[1]
            
            # Segment into 4-second windows (non-overlapping)
            n_windows = trial_len // window_size
            
            for win in range(n_windows):
                segment = eeg_data[:, win * window_size : (win + 1) * window_size]  # shape: (channels, 512)
                count += 1
                all_segments.append(segment)
                all_labels.append(side_dict[df[trial]])
            print(f"{pre_trial_idx}~{count}")

            # write trial info dict
            trial_info = {
                "subject_name": mat_files[subj].split('.')[0],
                "trial":trial,
                "attention_side": df[trial],
                "left_angel": left_angel,
                "right_angel": right_angel,
                "trial_start": pre_trial_idx,
                "trial_end": count,
            }
            all_trials.append(trial_info)
            pre_trial_idx = count 
        except:
            print(mat_files[subj], trial, 'load error')
    print(f"{mat_files[subj]} cumulated segments:{count}")



# Convert to numpy arrays
X = np.stack(all_segments)  # shape: (total_segments, channels, 512)
y = np.array(all_labels)    # shape: (total_segments,)

print("Data shape:", X.shape)
print("Labels shape:", y.shape)

n_epochs, n_channels, n_times = X.shape
sfreq = 128  # sampling frequency

# 建立 channel info
# 你給的通道名稱
ch_names = [
    'Cz', 'Fz', 'Fp1', 'F7', 'F3', 'FC1', 'C3', 'FC5', 'FT9', 'T7', 'CP5', 'CP1',
    'P3', 'P7', 'PO9', 'O1', 'Pz', 'Oz', 'O2', 'PO10', 'P8', 'P4', 'CP2', 'CP6',
    'T8', 'FT10', 'FC6', 'C4', 'FC2', 'F4', 'F8', 'Fp2'
]
ch_types = ['eeg'] * n_channels
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

# 建立 event array
# MNE 的 events shape 是 (n_epochs, 3)，通常中間值無用
# 最後一欄是 class label，需要轉換成 event id 後對應 dict
events = np.column_stack((
    np.arange(len(y)),  # arbitrary sample indices
    np.zeros(len(y), dtype=int),  # filler
    y
))

# 建立 event_id 字典
event_id = {f'class_{i}': i for i in np.unique(y)}

# 轉成 EpochsArray
epochs = mne.EpochsArray(X, info, events=events, event_id=event_id, verbose=False)
montage = mne.channels.make_standard_montage('standard_1020')
epochs.set_montage(montage)

# 檢查一下
print(epochs)
# epochs.plot(n_epochs=5, n_channels=10)

['azimuthmat.mat', 'expinfomat.mat', 'S02.mat', 'S03.mat', 'S04.mat', 'S06.mat', 'S07.mat', 'S09.mat', 'S12.mat', 'S13.mat', 'S15.mat', 'S16.mat', 'S17.mat', 'S18.mat', 'S19.mat', 'S21.mat', 'S22.mat', 'S23.mat', 'S25.mat', 'S26.mat', 'S27.mat']
S02.mat-0: -90.0/90.0
0~40
S02.mat-1: -90.0/90.0
40~60
S02.mat-2: -135.0/135.0
60~110
S02.mat-3: -135.0/135.0
110~140
S02.mat-4: 30.0/60.0 not the same angle 
S02.mat-5: 30.0/60.0 not the same angle 
S02.mat-6: 0.0/0.0
140~200
S02.mat-7: 0.0/0.0
200~210
S02.mat-8: -30.0/-90.0 not the same angle 
S02.mat-9: -30.0/-90.0 not the same angle 
S02.mat-10: 30.0/90.0 not the same angle 
S02.mat-11: 30.0/90.0 not the same angle 
S02.mat-12: -120.0/120.0
210~310
S02.mat-13: -120.0/120.0
310~380
S02.mat-14: -15.0/15.0
380~450
S02.mat-15: -15.0/15.0
450~500
S02.mat-16: -30.0/-60.0 not the same angle 
S02.mat-17: -30.0/-60.0 not the same angle 
S02.mat-18: -45.0/45.0
500~550
S02.mat-19: -45.0/45.0
550~600
S02.mat-20: -60.0/60.0
600~690
S02.mat-21: -60.0/60.

### Subject Group Training set and testing set

In [2]:
group_ = [(2, 7), (8, 15), (16, 19), (21, 23), (25, 27)]  # 5 groups
group_index_list = [[] for _ in range(len(group_))]      # list of 5 empty lists

cum_trials = 0 
for idx, info_dict in enumerate(all_trials):
    # print(f"{idx}-{info_dict['subject_name']}/{info_dict['trial']} - cumulated trials rate:{info_dict['trial_end']/count}")
    if choice_direction != 0:
        if abs(info_dict['left_angel']) != choice_direction:
            continue

    subject_int = int(info_dict['subject_name'].replace('S', ''))

    # 分配到對應的 group
    for group_id, (low, high) in enumerate(group_):
        if low <= subject_int <= high:
            trial_range = range(info_dict['trial_start'], info_dict['trial_end'])
            print(info_dict['subject_name'], info_dict['trial'], info_dict['trial_start'], info_dict['trial_end'])
            cum_trials += len(trial_range)
            group_index_list[group_id].extend(trial_range)
            break
print(f"Total trials: {cum_trials}")
cum_ratio = 0
for idx, g_l in enumerate(group_index_list):
    trial_ratio = len(g_l)/cum_trials
    cum_ratio += trial_ratio
    print(f"Group {idx+1}:(S{group_[idx][0]:02d}~S{group_[idx][1]:02d}) | total trials:{len(g_l)}, trials ratio:{trial_ratio*100:.1f}%, cumulated ratio:{cum_ratio*100:.1f}%")
    # print(len(g_l), trial_ratio, cum_ratio)


S02 0 0 40
S02 1 40 60
S03 14 1500 1580
S03 15 1580 1670
S04 0 2040 2100
S04 1 2100 2170
S06 0 3150 3210
S06 1 3210 3320
S07 20 5710 5770
S07 21 5770 5840
S09 10 6340 6410
S09 11 6410 6490
S12 12 7270 7330
S12 13 7330 7380
S13 0 7730 7790
S13 1 7790 7870
S15 8 9370 9470
S15 9 9470 9580
S16 16 11220 11290
S16 17 11290 11350
S17 22 13010 13080
S17 23 13080 13170
S18 16 14340 14440
S18 17 14440 14530
S19 6 15120 15210
S19 7 15210 15310
S21 18 17980 18060
S21 19 18060 18120
S21 24 18430 18530
S21 25 18530 18620
S21 26 18620 18720
S21 27 18720 18820
S21 30 19030 19120
S21 31 19120 19160
S22 6 19590 19660
S22 7 19660 19740
S22 8 19740 19830
S22 9 19830 19890
S22 19 20380 20480
S22 26 20950 21000
S22 27 21000 21080
S23 0 21080 21160
S23 1 21160 21250
S23 14 21780 21810
S23 15 21810 21850
S23 20 22040 22070
S23 28 22250 22260
S23 29 22260 22280
S25 2 22500 22550
S25 3 22550 22620
S25 4 22620 22690
S25 5 22690 22740
S25 24 23580 23590
S25 25 23590 23640
S25 30 23740 23750
S25 31 23750 23770
S26

### Step 2 - CSP and Log Power

In [3]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm, tree
from mne.decoding import CSP

def do_csp(bands, train_epochs, test_epochs, train_y, test_y):
    csp_list = []
    train_features = []
    test_features = []

    for name, (l_freq, h_freq) in bands.items():
        train_epochs_band = train_epochs.copy().filter(l_freq, h_freq, fir_design='firwin').get_data(copy=False)
        test_epochs_band = test_epochs.copy().filter(l_freq, h_freq, fir_design='firwin').get_data(copy=False)

        csp = CSP(n_components=4, reg='ledoit_wolf')
        train_csp = csp.fit_transform(train_epochs_band, train_y)
        test_csp = csp.transform(test_epochs_band)

        csp_list.append(csp)
        train_features.append(train_csp)
        test_features.append(test_csp)

    train_CSP_features = np.concatenate(train_features, axis=1)
    test_CSP_features = np.concatenate(test_features, axis=1)

    cv = ShuffleSplit(10, test_size=0.2, random_state=42)

    # Define classifiers
    lda = LinearDiscriminantAnalysis()
    svc = svm.SVC()
    dt = tree.DecisionTreeClassifier()

    # Cross-validation scores (optional, not used in return)
    _ = cross_val_score(svc, train_CSP_features, train_y, cv=cv)
    _ = cross_val_score(lda, train_CSP_features, train_y, cv=cv)
    _ = cross_val_score(dt, train_CSP_features, train_y, cv=cv)

    # Fit and predict
    svc.fit(train_CSP_features, train_y)
    lda.fit(train_CSP_features, train_y)
    dt.fit(train_CSP_features, train_y)

    svm_pred_y = svc.predict(test_CSP_features)
    lda_pred_y = lda.predict(test_CSP_features)
    dt_pred_y = dt.predict(test_CSP_features)

    return svm_pred_y, lda_pred_y, dt_pred_y, csp_list


### Step 3 - Do Leave one group out

In [4]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def leaveonegroupout(group_index, epochs, y, bands):
    pred_svm = []
    pred_lda = []
    pred_dt = []
    true_labels = []
    csp_list = []

    acc_svm_list, acc_lda_list, acc_dt_list = [], [], []
    f1_svm_list, f1_lda_list, f1_dt_list = [], [], []

    for valid_group in range(len(group_index)):
        print(f"--------------------------------------------------------------------------------------------------------------\n\n")
        print(f"\n\tGroup:{valid_group}\n")
        print(f"--------------------------------------------------------------------------------------------------------------\n\n")

        test_idx = group_index[valid_group]
        train_idx = [i for g, indices in enumerate(group_index) if g != valid_group for i in indices]

        train_epochs = epochs[train_idx].copy()
        test_epochs = epochs[test_idx].copy()
        train_y = y[train_idx]
        test_y = y[test_idx]

        temp_svm_pred_y, temp_lda_pred_y, temp_dt_pred_y, temp_csp_list = do_csp(bands, train_epochs, test_epochs, train_y, test_y)

        pred_svm.extend(temp_svm_pred_y)
        pred_lda.extend(temp_lda_pred_y)
        pred_dt.extend(temp_dt_pred_y)
        csp_list.extend(temp_csp_list)
        true_labels.extend(test_y)

        # Accuracy
        acc_svm = accuracy_score(test_y, temp_svm_pred_y)
        acc_lda = accuracy_score(test_y, temp_lda_pred_y)
        acc_dt = accuracy_score(test_y, temp_dt_pred_y)

        # F1
        f1_svm = f1_score(test_y, temp_svm_pred_y, average='macro')
        f1_lda = f1_score(test_y, temp_lda_pred_y, average='macro')
        f1_dt = f1_score(test_y, temp_dt_pred_y, average='macro')

        # Append to lists
        acc_svm_list.append(acc_svm)
        acc_lda_list.append(acc_lda)
        acc_dt_list.append(acc_dt)

        f1_svm_list.append(f1_svm)
        f1_lda_list.append(f1_lda)
        f1_dt_list.append(f1_dt)

    # Global metrics
    pred_svm = np.array(pred_svm)
    pred_lda = np.array(pred_lda)
    pred_dt = np.array(pred_dt)
    true_labels = np.array(true_labels)

    acc_svm_total = accuracy_score(true_labels, pred_svm)
    acc_lda_total = accuracy_score(true_labels, pred_lda)
    acc_dt_total = accuracy_score(true_labels, pred_dt)

    f1_svm_total = f1_score(true_labels, pred_svm, average='macro')
    f1_lda_total = f1_score(true_labels, pred_lda, average='macro')
    f1_dt_total = f1_score(true_labels, pred_dt, average='macro')

    # Std
    std_acc_svm = np.std(acc_svm_list)
    std_acc_lda = np.std(acc_lda_list)
    std_acc_dt = np.std(acc_dt_list)

    std_f1_svm = np.std(f1_svm_list)
    std_f1_lda = np.std(f1_lda_list)
    std_f1_dt = np.std(f1_dt_list)

    # Print results
    print(f"[SVM] Accuracy: {acc_svm_total:.4f}, Group-std: {std_acc_svm:.4f}, F1: {f1_svm_total:.4f}, Group-std: {std_f1_svm:.4f}")
    print(acc_svm_list)

    print(f"[LDA] Accuracy: {acc_lda_total:.4f}, Group-std: {std_acc_lda:.4f}, F1: {f1_lda_total:.4f}, Group-std: {std_f1_lda:.4f}")
    print(acc_lda_list)

    print(f"[DT ] Accuracy: {acc_dt_total:.4f}, Group-std: {std_acc_dt:.4f}, F1: {f1_dt_total:.4f}, Group-std: {std_f1_dt:.4f}")
    print(acc_dt_list)
    
    return acc_svm_total, f1_svm, acc_lda_total, f1_lda, acc_dt_total, f1_dt, csp_list


### RUN FB CSP Classification 

In [5]:
_, _, _, _, _, _, csp_list = leaveonegroupout(group_index_list, epochs, y, bands)

--------------------------------------------------------------------------------------------------------------



	Group:0

--------------------------------------------------------------------------------------------------------------


Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 141 samples (1.102 s)



C:\Users\richi\AppData\Local\Temp\ipykernel_68532\998527448.py:12: RuntimeWarning: filter_length (141) is longer than the signal (128), distortion is likely. Reduce filter length or filter a longer signal.
  train_epochs_band = train_epochs.copy().filter(l_freq, h_freq, fir_design='firwin').get_data(copy=False)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 141 samples (1.102 s)



[Parallel(n_jobs=1)]: Done 131200 out of 131200 | elapsed:   30.7s finished
C:\Users\richi\AppData\Local\Temp\ipykernel_68532\998527448.py:13: RuntimeWarning: filter_length (141) is longer than the signal (128), distortion is likely. Reduce filter length or filter a longer signal.
  test_epochs_band = test_epochs.copy().filter(l_freq, h_freq, fir_design='firwin').get_data(copy=False)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Computing rank from data with rank=None


...

Setting up band-pass filter from 18 - 22 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 18.00
- Lower transition bandwidth: 4.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Upper passband edge: 22.00 Hz
- Upper transition bandwidth: 5.50 Hz (-6 dB cutoff frequency: 24.75 Hz)
- Filter length: 95 samples (0.742 s)



[Parallel(n_jobs=1)]: Done 109760 out of 109760 | elapsed:   22.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Computing rank from data with rank=None


[Parallel(n_jobs=1)]: Done 42560 out of 42560 | elapsed:    6.0s finished


    Using tolerance 60 (2.2e-16 eps * 32 dim * 8.4e+15  max singular value)
    Estimated rank (data): 32
    data: rank 32 computed from 32 data channels with 0 projectors
Reducing data rank from 32 -> 32
Estimating class=0 covariance using LEDOIT_WOLF
Done.
Estimating class=1 covariance using LEDOIT_WOLF
Done.
[SVM] Accuracy: 0.5769, Group-std: 0.0684, F1: 0.5687, Group-std: 0.0647
[0.5772727272727273, 0.6672131147540984, 0.4626865671641791, 0.5543624161073826, 0.6180451127819548]
[LDA] Accuracy: 0.5662, Group-std: 0.0321, F1: 0.5649, Group-std: 0.0358
[0.55, 0.6065573770491803, 0.5253731343283582, 0.5449664429530201, 0.6]
[DT ] Accuracy: 0.5679, Group-std: 0.0553, F1: 0.5659, Group-std: 0.0520
[0.5303030303030303, 0.6442622950819672, 0.5671641791044776, 0.5, 0.6278195488721805]


### Plot CSP

In [ ]:
for idx, csp in enumerate(csp_list):
    # print(f"Filter Bank band: {bands[str(idx+1)]}")
    csp.plot_patterns(epochs.info, ch_type="eeg", units="Patterns (AU)", size=1.5)

### Mulit-class CSP
- 